# Trying to Semi-Brute Force P(J4,K5)

### Strategy: Find P(J_4, K_5, 8), then increase n by 1 by adding a new vertex in 2^n ways, reducing up to isomorphism, and then checking if it is in P(J_4,K_5)

#### imports and useful functions

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from itertools import permutations, combinations
import pynauty as pn
import time
import gc

def powerset(iterable,upper_bound=None, lower_bound=0):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    "upper bound is the largest cardinality you want your sets to be"
    s = list(iterable)
    if upper_bound is not None:
      up=upper_bound
    else:
      up=len(s)
    list1=[list(combinations(s, r)) for r in range(lower_bound,up+1)]
    list2=[]
    for mini in list1:
      list2=list2+mini
    return list2

def max_length(list1):
  max1=0
  for i in list1:
    long=len(i)
    if long>max1:
      max1=long
  return max1

def ContainsJ5(graph):
  #finds maximal cliques of the graph
  all_cliques=list(nx.enumerate_all_cliques(graph))
  max_length_cliques=max_length(all_cliques)
  if max_length_cliques>4:
    #means it contains a k5, so we done
    return True
  elif max_length_cliques<4:
    #too small
    return False
  elif max_length_cliques==4:
    max_cliques=[c for c in all_cliques if len(c)==4]
    for cliqueA in max_cliques:
      for cliqueB in max_cliques:
        #checks if 2 4-cliques are only different by one vertex
        union=set(cliqueA).union(set(cliqueB))
        if len(union)==5:
          return True
    return False
  else:
    print("the fuck happened?")

def ContainsJ4(graph):
  #finds maximal cliques of the graph
  all_cliques=list(nx.enumerate_all_cliques(graph))
  max_length_cliques=max_length(all_cliques)
  if max_length_cliques>3:
    #means it contains a k4, so we done
    return True
  elif max_length_cliques<3:
    #too small, j4 has triangles
    return False
  elif max_length_cliques==3:
    max_cliques=[c for c in all_cliques if len(c)==3]
    for cliqueA in max_cliques:
      for cliqueB in max_cliques:
        #checks if 2 3-cliques are only different by one vertex
        union=set(cliqueA).union(set(cliqueB))
        if len(union)==4:
          return True
    return False
  else:
    print("the fuck happened?")

def ContainsJ3(graph):
  #finds maximal cliques of the graph
  all_cliques=list(nx.enumerate_all_cliques(graph))
  max_length_cliques=max_length(all_cliques)
  if max_length_cliques>2:
    #means it contains a k3, so we done
    return True
  elif max_length_cliques<2:
    #too small
    return False
  elif max_length_cliques==2:
    max_cliques=[c for c in all_cliques if len(c)==2]
    for cliqueA in max_cliques:
      for cliqueB in max_cliques:
        #checks if 2 2-cliques are only different by one vertex
        union=set(cliqueA).union(set(cliqueB))
        if len(union)==3:
          return True
    return False
  else:
    print("the fuck happened?")

def convertNetworkXGraph(nx_graph):
    pn_graph = pn.Graph(len(nx_graph))
    for n in nx_graph:
        list_neighbors = []
        for nbs in nx_graph[n]:
            list_neighbors.append(nbs)
        pn_graph.connect_vertex(n, list_neighbors)
    return pn_graph

def ContainsAntiK5(graph):
  max_IS=max_length(list(nx.find_cliques(nx.complement(graph))))
  return max_IS>=5

def Up_to_isomorphism(list_of_nx_graphs):
    #takes a list of nx graphs and removes non-isomorphic copies
    Output=[]
    hash_set=set()
    for nx_graph in list_of_nx_graphs:
        pn_graph = convertNetworkXGraph(nx_graph)
        cert = pn.certificate(pn_graph)
        if cert not in hash_set:
            hash_set.add(cert)
            Output.append(nx_graph)
    return Output

def JustPj4k5(list_of_graphs):
  #takes in a list of graphs, outputs the ones in Pj4k5
  output=[]
  for current_graph in list_of_graphs:
    if not ContainsJ4(current_graph) and not ContainsAntiK5(current_graph):
      output+=[current_graph.copy()]
  return output


  

  
  


def Write_it_up(n, list1):
  #Writes up the things we just found
  with open("Pj4k5n"+str(n)+".txt","wb") as f:
    for graph in list1:
      nx.write_graph6(graph,f,header=False) 
  

test_graph=nx.complete_graph(4)
test_graph.remove_edge(1,2)
print(ContainsJ4(test_graph))

True


#### (n=1-7) Finding the easy cases 


In [25]:
Pj4k5_n1_to_n7=[]
#using the graph atlas for 1-7
for n in range(1,1253):
  #using the graph atlas for 1-7
  graph=nx.graph_atlas(n)
  if not ContainsAntiK5(graph) and not ContainsJ4(graph):
    Pj4k5_n1_to_n7+=[graph.copy()]
print(len(Pj4k5_n1_to_n7))

with open("Pj4k5n1to7.txt","wb") as f:
  for graph in Pj4k5_n1_to_n7:
    nx.write_graph6(graph,f,header=False) 

310


#### (n=8) Finding the last easier case 

In [26]:
# import the list of graphs with 8 vertices from txt file
Graphs_w_8v=[]
Graphs_w_8v=nx.read_graph6("Graphs_w_8v_saved.txt")
print(len(Graphs_w_8v))
Pj4k5_n8=JustPj4k5(Graphs_w_8v)
print(len(Pj4k5_n8))

with open("Pj4k5n8.txt","wb") as f:
  for graph in Pj4k5_n8:
    nx.write_graph6(graph,f,header=False) 

12346
897


#### (n=9) Trying the first harder Case 

First, some theory behind what I'm doing. I am currently trying to find all of P(J_4, K_5, 9). I have P(J_4, K_5, 8). Notice that if I have a graph G in P(J_4, K_5, 9) and I remove a vertex, then it couldn't have added a J_4 or a Anti-K_5. Thus, this subgraph would be in P(J_4, K_5, 8). This means that if I look at all the ways to add one vertex to every graph in P(J_4, K_5, 8), then we will have a superset of P(J_4, K_5, 9). Then we can just reduce up to isomorphism, and then further reduce by taking out the graphs with a J4 or Anti-K5. And then we stay winning. 


In [ ]:
#ways to connect the G8 and the new guy
n=9
Possible_Edges=[(n-1,k) for k in range(n-1)]
print(Possible_Edges)
G8s_to_check=Pj4k5_n8.copy()
combos_to_check=powerset(Possible_Edges)
possible_guys=[]
for combo in combos_to_check:
    for G8 in G8s_to_check:
        G9=G8.copy()
        G9.add_node(8)
        G9.add_edges_from(combo)
        possible_guys.append(G9.copy())
print(len(possible_guys))

possible_unique_guys=Up_to_isomorphism(possible_guys)
print(len(possible_unique_guys))

Pj4k5_n9=JustPj4k5(possible_unique_guys)
print(len(Pj4k5_n9))

with open("Pj4k5n9.txt","wb") as f:
  for graph in Pj4k5_n9:
    nx.write_graph6(graph,f,header=False) 


[(8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7)]
229632
74332
4463


#### (n=10) We keep it going 

Keep the train rolling.

Also, in the P(J3, K5) case I did the power set of the possible edges. However, for this vertex v we are adding, the degree has to be less than or equal to 8, so we will ignore the sets that give v more than degree 8.

This case gets too hard from the start, bc my computer can't handle the roughly 2 million initial cases before reducing up to isomorphisms. Idea #1: Reduce up to isomorphism as we work. So we hash them in as we run through the 2 million cases, which reduces the amount of set bullshit we have to do. This worked for this case which is good:)

In [2]:
#Load in n=9
#restarted the kernel to clear cache
Pj4k5_n9=nx.read_graph6("Pj4k5n9.txt")

In [3]:
import time
n=10
Possible_Edges=[(n-1,k) for k in range(n-1)]
G9s_to_check=Pj4k5_n9.copy()
combos_to_check=powerset(Possible_Edges,upper_bound=8)
print(len(G9s_to_check))
print(len(combos_to_check))
unique_possible_guys=[]
hash_set=set()
count=0
time1=time.time()
for combo in combos_to_check:
    for G9 in G9s_to_check:
        G10=G9.copy()
        G10.add_node(n-1)
        G10.add_edges_from(combo)
        #create the graph, then check if it's unique
        pn_graph = convertNetworkXGraph(G10)
        cert = pn.certificate(pn_graph)
        if cert not in hash_set:
            hash_set.add(cert)
            unique_possible_guys.append(G10)
        count+=1
        if count%100000==0:
            print(int(count/100000), len(unique_possible_guys),str(int(10*(time.time()-time1))/10) +"s per 100,000")
            time1=time.time()
print(len(unique_possible_guys))
time1=time.time()
Pj4k5_n10=JustPj4k5(unique_possible_guys)
print(len(Pj4k5_n10),str(int(10*(time.time()-time1))/10)+"s to narrow down to P(J4, K5)")

4463
511
1 65589 10.8s per 100,000
2 93512 9.5s per 100,000
3 142511 9.4s per 100,000
4 178531 9.6s per 100,000
5 206638 10.3s per 100,000
6 234192 10.1s per 100,000
7 291281 9.1s per 100,000
8 327264 10.8s per 100,000
9 365123 9.1s per 100,000
10 388710 10.7s per 100,000
11 409224 8.9s per 100,000
12 456150 9.3s per 100,000
13 511567 11.8s per 100,000
14 557413 9.1s per 100,000
15 598036 13.2s per 100,000
16 630672 9.1s per 100,000
17 655914 9.1s per 100,000
18 731575 16.4s per 100,000
19 797141 10.4s per 100,000
20 856871 9.7s per 100,000
21 914541 9.8s per 100,000
22 993469 21.8s per 100,000
1056113
23577 103.3s to narrow down to P(J4, K5)


In [4]:
# saving n=10 in a txt file
Write_it_up(10,Pj4k5_n10)

#### (n=11) We just keep trying

n=11,12 are probably going to be the hardest cases because there will be the most cases to deal with. 

Hopefully the size of P(J4,K5,n) peaks at n=10 or 11

Let's see what happens. 
Trial 1: Didn't go fast enough, and used 8 GB of RAM (I have 8 GB of RAM). The RAM limitations were slowing the computer down a lot. We also have 23 million cases, so we need another way to make the plan better. 

What if I test if the hash is unique, and if it is, then we test if it is in P(J4, K5, 11). The hash set will make sure we aren't overtesting, and the test if they are in P(J4, K5,11) means we don't store graphs we will just eliminate later.


In [2]:
#Load in n=10
#restarted the kernel to clear cache
Pj4k5_n10=nx.read_graph6("Pj4k5n10.txt")



In [ ]:
import time
n=11
Possible_Edges=[(n-1,k) for k in range(n-1)]
G10s_to_check=Pj4k5_n10.copy()
combos_to_check=powerset(Possible_Edges,upper_bound=8)
print(len(G10s_to_check))
print(len(combos_to_check))
Pj4k5_n11=[]
hash_set=set()
count=0
time1=time.time()
for combo in combos_to_check:
    for G10 in G10s_to_check:
        G11=G10.copy()
        G11.add_node(n-1)
        G11.add_edges_from(combo)
        #created the graph, then check if it's unique
        pn_graph = convertNetworkXGraph(G11)
        cert = pn.certificate(pn_graph)
        if cert not in hash_set:
            if not ContainsJ4(G11) and not ContainsAntiK5(G11):
                #We filter now so we aren't storing too much.
                Pj4k5_n11.append(G11)
                #fuck it, let's only filter the isomorphic copies of things in the set we want
                #this will slow things down, but make it less memory intensive
                hash_set.add(cert)
        count+=1
        if count%1000000==0:
            print(int(count/1000000),len(Pj4k5_n11),str(int(10*(time.time()-time1))/10) +"s per 1,000,000")
            time1=time.time()
print(len(Pj4k5_n11))


23577
1013
1 775601 39610 306.0s per 1,000,000
2 1348431 75537 198.0s per 1,000,000
3 1840922 91377 165.4s per 1,000,000
4 2156080 96955 138.0s per 1,000,000
5 2834179 97570 170.1s per 1,000,000
6 3376951 97670 160.3s per 1,000,000
7 3895301 97738 156.3s per 1,000,000
8 4307494 97771 155.5s per 1,000,000
9 4582969 97795 132.8s per 1,000,000
10 5313944 97796 180.0s per 1,000,000
11 5974853 97796 167.2s per 1,000,000
12 6519721 97796 159.4s per 1,000,000
13 7100285 97796 160.7s per 1,000,000
14 7533774 97796 146.1s per 1,000,000
15 7891910 97796 139.5s per 1,000,000
16 8740147 97796 197.8s per 1,000,000
17 9525363 97796 184.8s per 1,000,000
18 10240691 97796 181.2s per 1,000,000
19 10971153 97796 198.8s per 1,000,000
20 11586118 97796 191.1s per 1,000,000
21 12499974 97796 203.1s per 1,000,000
22 13356785 97796 199.1s per 1,000,000
23 14196397 97796 197.6s per 1,000,000
97796


In [ ]:
Write_it_up(11, Pj4k5_n11)

#### (n=12) Can we get 12???

I am gonna start by just seeing if I can do what I did last time and see how badly it goes. Although, I do get to have a lower bound on the degree now so that's exciting
first run checked 75 mil cases and didn't find a single graph from P(J4, K5). Also took 15 GB of RAM so that crashed my shit. 
Trial 2: Group the graphs by number of edges, and then we can run through each set of graphs with that many edges



In [2]:
#Load in n=11
#restarted the kernel to clear cache
Pj4k5_n11=nx.read_graph6("Pj4k5n11.txt")



In [3]:
#Sort Pj4k5_n11 by edge count

#First, we need to identify the possible edge counts for n=11
eleven_hash=set()
for graph in Pj4k5_n11:
    edge_count=len(graph.edges())
    if edge_count not in eleven_hash:
        eleven_hash.add(edge_count)
print(eleven_hash)
#Next, we sort into a dictionary by edge count
Pj4k5_n11_dict={k:[] for k in eleven_hash}
for graph in Pj4k5_n11:
    k=len(graph.edges())
    Pj4k5_n11_dict[k].append(graph)



{10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25}


In [5]:
#Now, sort the powerset by how many edges it adds to the G12
n=12
Possible_Edges=[(n-1,k) for k in range(n-1)]
powerset12=powerset(Possible_Edges,upper_bound=8,lower_bound=1)
#sort into a dictionary by # of edges added
powerset_dict={k:[] for k in range(1,9)} 
for edges in powerset12:
    powerset_dict[len(edges)].append(edges)

#There are 10-25 graphs in the G11, and 1-8 being added, so we will be checking graphs with 11 to 33 edges
#for each possible number n for e(G12), we will only check graphs constructed with numbers that sum to n

#Now, for each possible edge count for a G12, we will make a list of tuples 
# that indicated what combos add to that edge count
#so for 11 we have [(10,1)], for 12 we have [(10,2),(11,1)], etc
combos_dict={k:[] for k in range(11,34)}
for G11_size in eleven_hash:
    for edges_size in range(1,9):
        combos_dict[G11_size+edges_size].append((G11_size,edges_size))

print(combos_dict)

#Counting which cases will be the worst to do
for k in combos_dict:
    count3=0
    for tuple in combos_dict[k]:
        count3+=len(Pj4k5_n11_dict[tuple[0]])*len(powerset_dict[tuple[1]])
    print(k,count3)

{11: [(10, 1)], 12: [(10, 2), (11, 1)], 13: [(10, 3), (11, 2), (12, 1)], 14: [(10, 4), (11, 3), (12, 2), (13, 1)], 15: [(10, 5), (11, 4), (12, 3), (13, 2), (14, 1)], 16: [(10, 6), (11, 5), (12, 4), (13, 3), (14, 2), (15, 1)], 17: [(10, 7), (11, 6), (12, 5), (13, 4), (14, 3), (15, 2), (16, 1)], 18: [(10, 8), (11, 7), (12, 6), (13, 5), (14, 4), (15, 3), (16, 2), (17, 1)], 19: [(11, 8), (12, 7), (13, 6), (14, 5), (15, 4), (16, 3), (17, 2), (18, 1)], 20: [(12, 8), (13, 7), (14, 6), (15, 5), (16, 4), (17, 3), (18, 2), (19, 1)], 21: [(13, 8), (14, 7), (15, 6), (16, 5), (17, 4), (18, 3), (19, 2), (20, 1)], 22: [(14, 8), (15, 7), (16, 6), (17, 5), (18, 4), (19, 3), (20, 2), (21, 1)], 23: [(15, 8), (16, 7), (17, 6), (18, 5), (19, 4), (20, 3), (21, 2), (22, 1)], 24: [(16, 8), (17, 7), (18, 6), (19, 5), (20, 4), (21, 3), (22, 2), (23, 1)], 25: [(17, 8), (18, 7), (19, 6), (20, 5), (21, 4), (22, 3), (23, 2), (24, 1)], 26: [(18, 8), (19, 7), (20, 6), (21, 5), (22, 4), (23, 3), (24, 2), (25, 1)], 27:

In [6]:
#run through 11-33
Pj4k5_n12=[]
count=0
start_time=time.time()
for k in combos_dict:
    print(chr(10)+"Now checking graphs with "+str(k)+" edges")
    #grab all pairs i,j that add to k
    combos_to_check=combos_dict[k]
    hash_set_k=set()
    time1=time.time()
    #counts the number of graphs with k vertices checked
    k_count=0
    How_many_to_check=0
    for tuple in combos_to_check:
        How_many_to_check+=len(Pj4k5_n11_dict[tuple[0]])*len(powerset_dict[tuple[1]])
    print("We have",How_many_to_check, "graphs to check")
    for tuple in combos_to_check:
        #all G11s with the desired i value
        print(tuple)
        G11s_to_check=Pj4k5_n11_dict[tuple[0]]
        #all edge sets with the desired j value
        edge_combos=powerset_dict[tuple[1]]
        #iterate through all combos of them
        for edges in edge_combos:
            for G11 in G11s_to_check:
                #create the graph
                G12=G11.copy()
                G12.add_node(n-1)
                G12.add_edges_from(edges)
                #count how many we checked
                count+=1
                k_count+=1
                pn_graph = convertNetworkXGraph(G12)
                cert = pn.certificate(pn_graph)
                if cert not in hash_set_k:
                    #if not already checked, add it and check if it is in J4 K5
                    hash_set_k.add(cert)
                    if not ContainsJ4(G12) and not ContainsAntiK5(G12):
                        #We store if it is in the magical set we are trying to find
                        Pj4k5_n12.append(G12)
    elapsedk=time.time()-time1
    elapsedtotal=time.time()-start_time
    print("done with graphs with "+str(k)+" edges")
    print(str(k_count)+" graphs completed in "+str(int(10*elapsedk)/10)+" seconds")
    print(str(len(hash_set_k))+" unique graphs checked")
    print(str(count)+" graphs total completed in "+ str(int(elapsedtotal/6)/10)+" minutes")
    print("Total in P(J4,K5,12) found:",len(Pj4k5_n12))
                







Now checking graphs with 11 edges
We have  11 graphs to check
(10, 1)
done with graphs with 11 edges
11 graphs completed in 0.0 seconds
2 unique graphs checked
11 graphs total completed in 0.0 minutes
Total in P(J4,K5,12) found: 0

Now checking graphs with 12 edges
We have  88 graphs to check
(10, 2)
(11, 1)
done with graphs with 12 edges
88 graphs completed in 0.0 seconds
14 unique graphs checked
99 graphs total completed in 0.0 minutes
Total in P(J4,K5,12) found: 1

Now checking graphs with 13 edges
We have  462 graphs to check
(10, 3)
(11, 2)
(12, 1)
done with graphs with 13 edges
462 graphs completed in 0.1 seconds
84 unique graphs checked
561 graphs total completed in 0.0 minutes
Total in P(J4,K5,12) found: 2

Now checking graphs with 14 edges
We have  2024 graphs to check
(10, 4)
(11, 3)
(12, 2)
(13, 1)
done with graphs with 14 edges
2024 graphs completed in 0.7 seconds
483 unique graphs checked
2585 graphs total completed in 0.0 minutes
Total in P(J4,K5,12) found: 6

Now checki

In [7]:
#saving this bs
Write_it_up(12,Pj4k5_n12)

#### (n=13) Oh dear. Let's make it happen. 

This is probably going to be the worst case (fr this time) because we have a lot of graphs to check. 
3784 edge combos
180813 G12s to check
684,196,392 graphs to check 

edge cases: 14 edges to 38 edges

Added a specific garbage collector call to make it speed up a little bit.

Took on the n=14-25 cases first and stored them, because they are too small to give me real issues. 

For n=26,27,28,29,30,31,32, I will have a helluva time calculating everything. 


##### Run these 3 cells before each edge case

In [2]:
#Load in n=12
#restarted the kernel to clear cache
Pj4k5_n12=nx.read_graph6("Pj4k5n12.txt")
import gc


In [3]:
#Sort Pj4k5_n12 by edge count

#First, we need to identify the possible edge counts for n=12
twelve_hash=set()
#Also find the 
for graph in Pj4k5_n12:
    edge_count=len(graph.edges())
    if edge_count not in twelve_hash:
        twelve_hash.add(edge_count)
print(twelve_hash)
#Next, we sort into a dictionary by edge count
Pj4k5_n12_dict={k:[] for k in twelve_hash}
for graph in Pj4k5_n12:
    k=len(graph.edges())
    Pj4k5_n12_dict[k].append(graph)


{12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}


In [4]:
#Now, sort the powerset by how many edges it adds to the G13
n=13
Possible_Edges=[(n-1,k) for k in range(n-1)]
#lower bound is now 2
powerset12=powerset(Possible_Edges,upper_bound=8,lower_bound=2)
#sort into a dictionary by # of edges added
powerset_dict={k:[] for k in range(2,9)} 
for edges in powerset12:
    powerset_dict[len(edges)].append(edges)

#There are 12-30 edged graphs in the G12, and 2-8 being added, so we will be checking graphs with 14 to 38 edges
#for each possible number n for e(G13), we will only check graphs constructed with numbers that sum to n

#Now, for each possible edge count for a G13, we will make a list of tuples 
# that indicated what combos add to that edge count
#so for 14 we have [(12,2)], for 15 we have [(13,2),(12,3)], etc
combos_dict={k:[] for k in range(14,39)}
for G12_size in twelve_hash:
    for edges_size in range(2,9):
        combos_dict[G12_size+edges_size].append((G12_size,edges_size))

print(combos_dict)

#Counting which cases will be the worst to do
for k in combos_dict:
    count3=0
    for tuple in combos_dict[k]:
        count3+=len(Pj4k5_n12_dict[tuple[0]])*len(powerset_dict[tuple[1]])
    print(k,count3)

{14: [(12, 2)], 15: [(12, 3), (13, 2)], 16: [(12, 4), (13, 3), (14, 2)], 17: [(12, 5), (13, 4), (14, 3), (15, 2)], 18: [(12, 6), (13, 5), (14, 4), (15, 3), (16, 2)], 19: [(12, 7), (13, 6), (14, 5), (15, 4), (16, 3), (17, 2)], 20: [(12, 8), (13, 7), (14, 6), (15, 5), (16, 4), (17, 3), (18, 2)], 21: [(13, 8), (14, 7), (15, 6), (16, 5), (17, 4), (18, 3), (19, 2)], 22: [(14, 8), (15, 7), (16, 6), (17, 5), (18, 4), (19, 3), (20, 2)], 23: [(15, 8), (16, 7), (17, 6), (18, 5), (19, 4), (20, 3), (21, 2)], 24: [(16, 8), (17, 7), (18, 6), (19, 5), (20, 4), (21, 3), (22, 2)], 25: [(17, 8), (18, 7), (19, 6), (20, 5), (21, 4), (22, 3), (23, 2)], 26: [(18, 8), (19, 7), (20, 6), (21, 5), (22, 4), (23, 3), (24, 2)], 27: [(19, 8), (20, 7), (21, 6), (22, 5), (23, 4), (24, 3), (25, 2)], 28: [(20, 8), (21, 7), (22, 6), (23, 5), (24, 4), (25, 3), (26, 2)], 29: [(21, 8), (22, 7), (23, 6), (24, 5), (25, 4), (26, 3), (27, 2)], 30: [(22, 8), (23, 7), (24, 6), (25, 5), (26, 4), (27, 3), (28, 2)], 31: [(23, 8), (

##### I need to run through 14-38, but it would perhaps be easier to run through some of the faster cases before tackling the hard cases (26-32). So first I'm gonna run 14-25, then try and tackle the worse cases. 

In [ ]:
#run through 14-25
Pj4k5_n13=[]
count=0
start_time=time.time()
for k in range(14,26):
    print(chr(10)+"Now checking graphs with "+str(k)+" edges")
    #grab all pairs i,j that add to k
    combos_to_check=combos_dict[k]
    hash_set_k=set()
    time1=time.time()
    #counts the number of graphs with k vertices checked
    k_count=0
    How_many_to_check=0
    for tuple in combos_to_check:
        How_many_to_check+=len(Pj4k5_n12_dict[tuple[0]])*len(powerset_dict[tuple[1]])
    print("We have",How_many_to_check, "graphs to check")
    for tuple in combos_to_check:
        #all G12s with the desired i value
        print(tuple)
        G12s_to_check=Pj4k5_n12_dict[tuple[0]]
        #all edge sets with the desired j value
        edge_combos=powerset_dict[tuple[1]]
        #iterate through all combos of them
        for edges in edge_combos:
            for G12 in G12s_to_check:
                #create the graph
                G13=G12.copy()
                G13.add_node(n-1)
                G13.add_edges_from(edges)
                #count how many we checked
                count+=1
                k_count+=1
                pn_graph = convertNetworkXGraph(G13)
                cert = pn.certificate(pn_graph)
                if cert not in hash_set_k:
                    #if not already checked, add it and check if it is in J4 K5
                    hash_set_k.add(cert)
                    if not ContainsJ4(G13) and not ContainsAntiK5(G13):
                        #We store if it is in the magical set we are trying to find
                        Pj4k5_n13.append(G13)
    elapsedk=time.time()-time1
    elapsedtotal=time.time()-start_time
    print("done with graphs with "+str(k)+" edges")
    print(str(k_count)+" graphs completed in "+str(int(10*elapsedk)/10)+" seconds")
    print(str(len(hash_set_k))+" unique graphs checked")
    print(str(count)+" graphs total completed in "+ str(int(elapsedtotal/6)/10)+" minutes")
    print("Total in P(J4,K5,13) found:",len(Pj4k5_n13))
    #clears the hash set so it frees up space
    del hash_set_k
    gc.collect()
                




Now checking graphs with 14 edges
We have 66 graphs to check
(12, 2)
done with graphs with 14 edges
66 graphs completed in 0.0 seconds
2 unique graphs checked
66 graphs total completed in 0.0 minutes
Total in P(J4,K5,13) found: 0

Now checking graphs with 15 edges
We have 286 graphs to check
(12, 3)
(13, 2)
done with graphs with 15 edges
286 graphs completed in 0.0 seconds
11 unique graphs checked
352 graphs total completed in 0.0 minutes
Total in P(J4,K5,13) found: 0

Now checking graphs with 16 edges
We have 979 graphs to check
(12, 4)
(13, 3)
(14, 2)
done with graphs with 16 edges
979 graphs completed in 0.1 seconds
65 unique graphs checked
1331 graphs total completed in 0.0 minutes
Total in P(J4,K5,13) found: 0

Now checking graphs with 17 edges
We have 3091 graphs to check
(12, 5)
(13, 4)
(14, 3)
(15, 2)
done with graphs with 17 edges
3091 graphs completed in 1.2 seconds
366 unique graphs checked
4422 graphs total completed in 0.1 minutes
Total in P(J4,K5,13) found: 0

Now checki

In [ ]:
#saving the partial census we have completed
#So we can clear things and deal with individual cases
with open("Pj4k5n13e14_25.txt","wb") as f:
    for graph in Pj4k5_n13:
      nx.write_graph6(graph,f,header=False) 

##### e=26 case, just keep moving. 

In [ ]:
#I reset the edge cases, now going to run the code for just e=26
Pj4k5_n13=[]
count=0
start_time=time.time()
k=26
print(chr(10)+"Now checking graphs with "+str(k)+" edges")
#grab all pairs i,j that add to k
combos_to_check=combos_dict[k]
hash_set_k=set()
time1=time.time()
#counts the number of graphs with k vertices checked
k_count=0
How_many_to_check=0
for tuple in combos_to_check:
    How_many_to_check+=len(Pj4k5_n12_dict[tuple[0]])*len(powerset_dict[tuple[1]])
print("We have",How_many_to_check, "graphs to check")
for tuple in combos_to_check:
    #all G12s with the desired i value
    print(tuple)
    G12s_to_check=Pj4k5_n12_dict[tuple[0]]
    #all edge sets with the desired j value
    edge_combos=powerset_dict[tuple[1]]
    #iterate through all combos of them
    for edges in edge_combos:
        for G12 in G12s_to_check:
            #create the graph
            G13=G12.copy()
            G13.add_node(n-1)
            G13.add_edges_from(edges)
            #count how many we checked
            count+=1
            k_count+=1
            pn_graph = convertNetworkXGraph(G13)
            cert = pn.certificate(pn_graph)
            if cert not in hash_set_k:
                 #if not already checked, add it and check if it is in J4 K5
                hash_set_k.add(cert)
                if not ContainsJ4(G13) and not ContainsAntiK5(G13):
                    #We store if it is in the magical set we are trying to find
                    Pj4k5_n13.append(G13)
elapsedk=time.time()-time1
elapsedtotal=time.time()-start_time
print("done with graphs with "+str(k)+" edges")
print(str(k_count)+" graphs completed in "+str(int(10*elapsedk)/10)+" seconds")
print(str(len(hash_set_k))+" unique graphs checked")
print(str(count)+" graphs total completed in "+ str(int(elapsedtotal/6)/10)+" minutes")
print("Total in P(J4,K5,13) found:",len(Pj4k5_n13))


Now checking graphs with 26 edges
We have 43972115 graphs to check
(18, 8)
(19, 7)
(20, 6)
(21, 5)
(22, 4)
(23, 3)
(24, 2)
done with graphs with 26 edges
43972115 graphs completed in 12430.3 seconds
35789966 unique graphs checked
43972115 graphs total completed in 207.1 minutes
Total in P(J4,K5,13) found: 4008


In [ ]:
#saving the partial census we have completed
#So we can clear things and deal with individual cases
with open("Pj4k5n13e26.txt","wb") as f:
    for graph in Pj4k5_n13:
      nx.write_graph6(graph,f,header=False) 

##### e=27 case, why not

I'm gonna restart the hash set each time we do a new a combo. It will result in some things being checked multiple times, but it will stop the thing from crashing, because the hash set is the thing getting too large. 

In [ ]:
#I reset the edge cases, now going to run the code for just n=27
Pj4k5_n13_e27=[]
count=0
start_time=time.time()
k=27
print(chr(10)+"Now checking graphs with "+str(k)+" edges")
#grab all pairs i,j that add to k
combos_to_check=combos_dict[k]
time1=time.time()
#counts the number of graphs with k vertices checked
k_count=0
How_many_to_check=0
for tuple in combos_to_check:
    How_many_to_check+=len(Pj4k5_n12_dict[tuple[0]])*len(powerset_dict[tuple[1]])
print("We have",How_many_to_check, "graphs to check")
for tuple in combos_to_check:
    #new hash set for each pair
    #only save the stuff already in the set
    hash_set_k=set(Pj4k5_n13_e27)
    #all G12s with the desired i value
    print(tuple)
    G12s_to_check=Pj4k5_n12_dict[tuple[0]]
    #all edge sets with the desired j value
    edge_combos=powerset_dict[tuple[1]]
    #iterate through all combos of them
    print(len(G12s_to_check)*len(edge_combos))
    for edges in edge_combos:
        for G12 in G12s_to_check:
            #create the graph
            G13=G12.copy()
            G13.add_node(n-1)
            G13.add_edges_from(edges)
            #count how many we checked
            count+=1
            k_count+=1
            pn_graph = convertNetworkXGraph(G13)
            cert = pn.certificate(pn_graph)
            if cert not in hash_set_k:
                 #if not already checked, add it and check if it is in J4 K5
                hash_set_k.add(cert)
                if not ContainsJ4(G13) and not ContainsAntiK5(G13):
                    #We store if it is in the magical set we are trying to find
                    Pj4k5_n13_e27.append(G13)
    #clear the hash set
    del hash_set_k
    gc.collect()
#get rid of duplicates
Pj4k5_n13_e27=Up_to_isomorphism(Pj4k5_n13_e27)
elapsedk=time.time()-time1
elapsedtotal=time.time()-start_time
print("done with graphs with "+str(k)+" edges")
print(str(k_count)+" graphs completed in "+str(int(10*elapsedk)/10)+" seconds")
print("Total in P(J4,K5,13,27) found:",len(Pj4k5_n13_e27))


Now checking graphs with 27 edges
We have 73943353 graphs to check
(19, 8)
793485
(20, 7)
3910896
(21, 6)
12519276
(22, 5)
23153328
(23, 4)
21961170
(24, 3)
9752380
(25, 2)
1852818
done with graphs with 27 edges
73943353 graphs completed in 19899.3 seconds
Total in P(J4,K5,13,27) found: 8774


In [ ]:
#saving the partial census we have completed
#So we can clear things and deal with individual cases
with open("Pj4k5n13e27.txt","wb") as f:
    for graph in Pj4k5_n13_e27:
      nx.write_graph6(graph,f,header=False) 

##### e=28 case

Same strategy as e=27.

If I can do this case, then I am reasonably confident I can do all of the cases

In [ ]:
#I reset the edge cases, now going to run the code for just n=28
Pj4k5_n13_e28=[]
count=0
start_time=time.time()
k=28
print(chr(10)+"Now checking graphs with "+str(k)+" edges")
#grab all pairs i,j that add to k
combos_to_check=combos_dict[k]
time1=time.time()
#counts the number of graphs with k vertices checked
k_count=0
How_many_to_check=0
for tuple in combos_to_check:
    How_many_to_check+=len(Pj4k5_n12_dict[tuple[0]])*len(powerset_dict[tuple[1]])
print("We have",How_many_to_check, "graphs to check")
for tuple in combos_to_check:
    #new hash set for each pair
    #only save the stuff already in the set
    hash_set_k=set(Pj4k5_n13_e28)
    #all G12s with the desired i value
    print(tuple)
    G12s_to_check=Pj4k5_n12_dict[tuple[0]]
    #all edge sets with the desired j value
    edge_combos=powerset_dict[tuple[1]]
    #iterate through all combos of them
    print(len(G12s_to_check)*len(edge_combos))
    for edges in edge_combos:
        for G12 in G12s_to_check:
            #create the graph
            G13=G12.copy()
            G13.add_node(n-1)
            G13.add_edges_from(edges)
            #count how many we checked
            count+=1
            k_count+=1
            pn_graph = convertNetworkXGraph(G13)
            cert = pn.certificate(pn_graph)
            if cert not in hash_set_k:
                 #if not already checked, add it and check if it is in J4 K5
                hash_set_k.add(cert)
                if not ContainsJ4(G13) and not ContainsAntiK5(G13):
                    #We store if it is in the magical set we are trying to find
                    Pj4k5_n13_e28.append(G13)
    #clear the hash set
    del hash_set_k
    gc.collect()
#get rid of duplicates(just in case)
Pj4k5_n13_e28=Up_to_isomorphism(Pj4k5_n13_e28)
elapsedk=time.time()-time1
elapsedtotal=time.time()-start_time
print("done with graphs with "+str(k)+" edges")
print(str(k_count)+" graphs completed in "+str(int(10*elapsedk)/10)+" seconds")
print("Total in P(J4,K5,13,28) found:",len(Pj4k5_n13_e28))


Now checking graphs with 28 edges
We have 104170121 graphs to check
(20, 8)
2444310
(21, 7)
10730808
(22, 6)
27012216
(23, 5)
35137872
(24, 4)
21942855
(25, 3)
6176060
(26, 2)
726000
done with graphs with 28 edges
104170121 graphs completed in 29677.3 seconds
Total in P(J4,K5,13,28) found: 12235


In [ ]:
#saving the partial census we have completed
#So we can clear things and deal with individual cases
with open("Pj4k5n13e28.txt","wb") as f:
    for graph in Pj4k5_n13_e28:
      nx.write_graph6(graph,f,header=False) 

##### e=29 case

This is the biggest case, so if we get past it, we are gonna get through this case. 

In [ ]:
#I reset the edge cases, now going to run the code for just n=29
Pj4k5_n13_e29=[]
count=0
start_time=time.time()
k=29
print(chr(10)+"Now checking graphs with "+str(k)+" edges")
#grab all pairs i,j that add to k
combos_to_check=combos_dict[k]
time1=time.time()
How_many_to_check=0
for tuple in combos_to_check:
    How_many_to_check+=len(Pj4k5_n12_dict[tuple[0]])*len(powerset_dict[tuple[1]])
print("We have",How_many_to_check, "graphs to check")
for tuple in combos_to_check:
    #new hash set for each pair
    hash_set_k=set()
    #all G12s with the desired i value
    print(tuple)
    G12s_to_check=Pj4k5_n12_dict[tuple[0]]
    #all edge sets with the desired j value
    edge_combos=powerset_dict[tuple[1]]
    #iterate through all combos of them
    print(len(G12s_to_check)*len(edge_combos))
    for edges in edge_combos:
        for G12 in G12s_to_check:
            #create the graph
            G13=G12.copy()
            G13.add_node(n-1)
            G13.add_edges_from(edges)
            #count how many we checked
            count+=1
            pn_graph = convertNetworkXGraph(G13)
            cert = pn.certificate(pn_graph)
            if cert not in hash_set_k:
                 #if not already checked, add it and check if it is in J4 K5
                hash_set_k.add(cert)
                if not ContainsJ4(G13) and not ContainsAntiK5(G13):
                    #We store if it is in the magical set we are trying to find
                    Pj4k5_n13_e29.append(G13)
    #clear the hash set
    del hash_set_k
    gc.collect()
#get rid of duplicates
Pj4k5_n13_e29=Up_to_isomorphism(Pj4k5_n13_e29)
elapsedk=time.time()-time1
elapsedtotal=time.time()-start_time
print("done with graphs with "+str(k)+" edges")
print(str(count)+" graphs completed in "+str(int(10*elapsedk)/10)+" seconds")
print("Total in P(J4,K5,13,29) found:",len(Pj4k5_n13_e29))


Now checking graphs with 29 edges
We have 122450504 graphs to check
(21, 8)
6706755
(22, 7)
23153328
(23, 6)
40994184
(24, 5)
35108568
(25, 4)
13896135
(26, 3)
2420000
(27, 2)
171534
done with graphs with 29 edges


NameError: name 'k_count' is not defined

In [ ]:
#The error above didn't break anything, it was just the print statement at the end that I messed up
#this is the fixed version 
print("done with graphs with "+str(k)+" edges")
print(str(count)+" graphs completed in "+str(int(10*elapsedk)/10)+" seconds")
print("Total in P(J4,K5,13,29) found:",len(Pj4k5_n13_e29))

done with graphs with 29 edges
122450504 graphs completed in 34104.3 seconds
Total in P(J4,K5,13,29) found: 10903


In [ ]:
#saving the partial census we have completed
#So we can clear things and deal with individual cases
with open("Pj4k5n13e29.txt","wb") as f:
    for graph in Pj4k5_n13_e29:
      nx.write_graph6(graph,f,header=False) 

##### e=30 case! We got this!

The largest case was that last e=29 case, so we are in the home stretch for n=13. 


In [ ]:
#I reset the edge cases, now going to run the code for just n=30
Pj4k5_n13_e30=[]
count=0
start_time=time.time()
k=30
print(chr(10)+"Now checking graphs with "+str(k)+" edges")
#grab all pairs i,j that add to k
combos_to_check=combos_dict[k]
time1=time.time()
How_many_to_check=0
for tuple in combos_to_check:
    How_many_to_check+=len(Pj4k5_n12_dict[tuple[0]])*len(powerset_dict[tuple[1]])
print("We have",How_many_to_check, "graphs to check")
for tuple in combos_to_check:
    #new hash set for each pair
    hash_set_k=set()
    #all G12s with the desired i value
    print(tuple)
    G12s_to_check=Pj4k5_n12_dict[tuple[0]]
    #all edge sets with the desired j value
    edge_combos=powerset_dict[tuple[1]]
    #iterate through all combos of them
    print(len(G12s_to_check)*len(edge_combos))
    for edges in edge_combos:
        for G12 in G12s_to_check:
            #create the graph
            G13=G12.copy()
            G13.add_node(n-1)
            G13.add_edges_from(edges)
            #count how many we checked
            count+=1
            pn_graph = convertNetworkXGraph(G13)
            cert = pn.certificate(pn_graph)
            if cert not in hash_set_k:
                 #if not already checked, add it and check if it is in J4 K5
                hash_set_k.add(cert)
                if not ContainsJ4(G13) and not ContainsAntiK5(G13):
                    #We store if it is in the magical set we are trying to find
                    Pj4k5_n13_e30.append(G13)
    #clear the hash set
    del hash_set_k
    gc.collect()
#get rid of duplicates
Pj4k5_n13_e30=Up_to_isomorphism(Pj4k5_n13_e30)
elapsedk=time.time()-time1
elapsedtotal=time.time()-start_time
print("done with graphs with "+str(k)+" edges")
print(str(count)+" graphs completed in "+str(int(10*elapsedk)/10)+" seconds")
print(f"Total in P(J4,K5,13,{str(k)}) found:",len(Pj4k5_n13_e30))


Now checking graphs with 30 edges
We have 118843384 graphs to check
(22, 8)
14470830
(23, 7)
35137872
(24, 6)
40959996
(25, 5)
22233816
(26, 4)
5445000
(27, 3)
571780
(28, 2)
24090
done with graphs with 30 edges
118843384 graphs completed in 34946.9 seconds
Total in P(J4,K5,13,30) found: 6246


In [ ]:
#saving the partial census we have completed
#So we can clear things and deal with individual cases
with open("Pj4k5n13e30.txt","wb") as f:
    for graph in Pj4k5_n13_e30:
      nx.write_graph6(graph,f,header=False) 

##### e=31 case



In [ ]:
#I reset the edge cases, now going to run the code for just n=31
Pj4k5_n13_e31=[]
count=0
start_time=time.time()
k=31
print(chr(10)+"Now checking graphs with "+str(k)+" edges")
#grab all pairs i,j that add to k
combos_to_check=combos_dict[k]
time1=time.time()
How_many_to_check=0
for tuple in combos_to_check:
    How_many_to_check+=len(Pj4k5_n12_dict[tuple[0]])*len(powerset_dict[tuple[1]])
print("We have",How_many_to_check, "graphs to check")
for tuple in combos_to_check:
    #new hash set for each pair
    hash_set_k=set()
    #all G12s with the desired i value
    print(tuple)
    G12s_to_check=Pj4k5_n12_dict[tuple[0]]
    #all edge sets with the desired j value
    edge_combos=powerset_dict[tuple[1]]
    #iterate through all combos of them
    print(len(G12s_to_check)*len(edge_combos))
    for edges in edge_combos:
        for G12 in G12s_to_check:
            #create the graph
            G13=G12.copy()
            G13.add_node(n-1)
            G13.add_edges_from(edges)
            #count how many we checked
            count+=1
            pn_graph = convertNetworkXGraph(G13)
            cert = pn.certificate(pn_graph)
            if cert not in hash_set_k:
                 #if not already checked, add it and check if it is in J4 K5
                hash_set_k.add(cert)
                if not ContainsJ4(G13) and not ContainsAntiK5(G13):
                    #We store if it is in the magical set we are trying to find
                    Pj4k5_n13_e31.append(G13)
    #clear the hash set
    del hash_set_k
    gc.collect()
#get rid of duplicates
Pj4k5_n13_e31=Up_to_isomorphism(Pj4k5_n13_e31)
elapsedk=time.time()-time1
elapsedtotal=time.time()-start_time
print("done with graphs with "+str(k)+" edges")
print(str(count)+" graphs completed in "+str(int(10*elapsedk)/10)+" seconds")
print(f"Total in P(J4,K5,13,{str(k)}) found:",len(Pj4k5_n13_e31))


Now checking graphs with 31 edges
We have 93089909 graphs to check
(23, 8)
21961170
(24, 7)
35108568
(25, 6)
25939452
(26, 5)
8712000
(27, 4)
1286505
(28, 3)
80300
(29, 2)
1914
done with graphs with 31 edges
93089909 graphs completed in 25342.7 seconds
Total in P(J4,K5,13,31) found: 2271


In [ ]:
#saving the partial census we have completed
#So we can clear things and deal with individual cases
with open("Pj4k5n13e31.txt","wb") as f:
    for graph in Pj4k5_n13_e31:
      nx.write_graph6(graph,f,header=False) 

##### e=32 case

This is the last one we need to do individually. 

In [ ]:
#I reset the edge cases, now going to run the code for just n=32
Pj4k5_n13_e32=[]
count=0
start_time=time.time()
k=32
print(chr(10)+"Now checking graphs with "+str(k)+" edges")
#grab all pairs i,j that add to k
combos_to_check=combos_dict[k]
time1=time.time()
How_many_to_check=0
for tuple in combos_to_check:
    How_many_to_check+=len(Pj4k5_n12_dict[tuple[0]])*len(powerset_dict[tuple[1]])
print("We have",How_many_to_check, "graphs to check")
for tuple in combos_to_check:
    #new hash set for each pair
    hash_set_k=set()
    #all G12s with the desired i value
    print(tuple)
    G12s_to_check=Pj4k5_n12_dict[tuple[0]]
    #all edge sets with the desired j value
    edge_combos=powerset_dict[tuple[1]]
    #iterate through all combos of them
    print(len(G12s_to_check)*len(edge_combos))
    for edges in edge_combos:
        for G12 in G12s_to_check:
            #create the graph
            G13=G12.copy()
            G13.add_node(n-1)
            G13.add_edges_from(edges)
            #count how many we checked
            count+=1
            pn_graph = convertNetworkXGraph(G13)
            cert = pn.certificate(pn_graph)
            if cert not in hash_set_k:
                 #if not already checked, add it and check if it is in J4 K5
                hash_set_k.add(cert)
                if not ContainsJ4(G13) and not ContainsAntiK5(G13):
                    #We store if it is in the magical set we are trying to find
                    Pj4k5_n13_e32.append(G13)
    #clear the hash set
    del hash_set_k
    gc.collect()
#get rid of duplicates
Pj4k5_n13_e32=Up_to_isomorphism(Pj4k5_n13_e32)
elapsedk=time.time()-time1
elapsedtotal=time.time()-start_time
print("done with graphs with "+str(k)+" edges")
print(str(count)+" graphs completed in "+str(int(10*elapsedk)/10)+" seconds")
print(f"Total in P(J4,K5,13,{str(k)}) found:",len(Pj4k5_n13_e32))


Now checking graphs with 32 edges
We have 56586398 graphs to check
(24, 8)
21942855
(25, 7)
22233816
(26, 6)
10164000
(27, 5)
2058408
(28, 4)
180675
(29, 3)
6380
(30, 2)
264
done with graphs with 32 edges
56586398 graphs completed in 15569.4 seconds
Total in P(J4,K5,13,32) found: 518


In [ ]:
#saving the partial census we have completed
#So we can clear things and deal with individual cases
with open("Pj4k5n13e32.txt","wb") as f:
    for graph in Pj4k5_n13_e32:
      nx.write_graph6(graph,f,header=False) 

##### e=33-38, last cases

Just finishing this out.

In [5]:
#run through 33-38
Pj4k5_n13_e33_38=[]
count=0
start_time=time.time()
for k in range(33,39):
    print(chr(10)+"Now checking graphs with "+str(k)+" edges")
    #grab all pairs i,j that add to k
    combos_to_check=combos_dict[k]
    hash_set_k=set()
    time1=time.time()
    #counts the number of graphs with k vertices checked
    k_count=0
    How_many_to_check=0
    for tuple in combos_to_check:
        How_many_to_check+=len(Pj4k5_n12_dict[tuple[0]])*len(powerset_dict[tuple[1]])
    print("We have",How_many_to_check, "graphs to check")
    for tuple in combos_to_check:
        #all G12s with the desired i value
        print(tuple)
        G12s_to_check=Pj4k5_n12_dict[tuple[0]]
        #all edge sets with the desired j value
        edge_combos=powerset_dict[tuple[1]]
        #iterate through all combos of them
        for edges in edge_combos:
            for G12 in G12s_to_check:
                #create the graph
                G13=G12.copy()
                G13.add_node(n-1)
                G13.add_edges_from(edges)
                #count how many we checked
                count+=1
                k_count+=1
                pn_graph = convertNetworkXGraph(G13)
                cert = pn.certificate(pn_graph)
                if cert not in hash_set_k:
                    #if not already checked, add it and check if it is in J4 K5
                    hash_set_k.add(cert)
                    if not ContainsJ4(G13) and not ContainsAntiK5(G13):
                        #We store if it is in the magical set we are trying to find
                        Pj4k5_n13_e33_38.append(G13)
    elapsedk=time.time()-time1
    elapsedtotal=time.time()-start_time
    print("done with graphs with "+str(k)+" edges")
    print(str(k_count)+" graphs completed in "+str(int(10*elapsedk)/10)+" seconds")
    print(str(len(hash_set_k))+" unique graphs checked")
    print(str(count)+" graphs total completed in "+ str(int(elapsedtotal/6)/10)+" minutes")
    print("Total in P(J4,K5,13,33-38) found:",len(Pj4k5_n13_e33_38))
    #clears the hash set so it frees up space
    del hash_set_k
    gc.collect()
                




Now checking graphs with 33 edges
We have 25313926 graphs to check
(25, 8)
(26, 7)
(27, 6)
(28, 5)
(29, 4)
(30, 3)
done with graphs with 33 edges
25313926 graphs completed in 6813.5 seconds
24311531 unique graphs checked
25313926 graphs total completed in 113.5 minutes
Total in P(J4,K5,13,33-38) found: 69

Now checking graphs with 34 edges
We have 7865616 graphs to check
(26, 8)
(27, 7)
(28, 6)
(29, 5)
(30, 4)
done with graphs with 34 edges
7865616 graphs completed in 2175.4 seconds
7440356 unique graphs checked
33179542 graphs total completed in 150.1 minutes
Total in P(J4,K5,13,33-38) found: 76

Now checking graphs with 35 edges
We have 1605549 graphs to check
(27, 8)
(28, 7)
(29, 6)
(30, 5)
done with graphs with 35 edges
1605549 graphs completed in 424.0 seconds
1455530 unique graphs checked
34785091 graphs total completed in 157.4 minutes
Total in P(J4,K5,13,33-38) found: 76

Now checking graphs with 36 edges
We have 207339 graphs to check
(28, 8)
(29, 7)
(30, 6)
done with graphs 

In [6]:
#saving the partial census we have completed
#So we can clear things and deal with individual cases
with open("Pj4k5n13e33_38.txt","wb") as f:
    for graph in Pj4k5_n13_e33_38:
      nx.write_graph6(graph,f,header=False) 

##### Combine it into one text file

I just want it to be available all one txt file. So I will load them all in and recombine them now. 

In [2]:
import networkx as nx
Pj4k5_n13_e14_25=nx.read_graph6("Pj4k5n13e14_25.txt")
Pj4k5_n13_e26=nx.read_graph6("Pj4k5n13e26.txt")
Pj4k5_n13_e27=nx.read_graph6("Pj4k5n13e27.txt")
Pj4k5_n13_e28=nx.read_graph6("Pj4k5n13e28.txt")
Pj4k5_n13_e29=nx.read_graph6("Pj4k5n13e29.txt")
Pj4k5_n13_e30=nx.read_graph6("Pj4k5n13e30.txt")
Pj4k5_n13_e31=nx.read_graph6("Pj4k5n13e31.txt")
Pj4k5_n13_e32=nx.read_graph6("Pj4k5n13e32.txt")
Pj4k5_n13_e33_38=nx.read_graph6("Pj4k5n13e33_38.txt")
Pj4k5_n13_final=Pj4k5_n13_e14_25+Pj4k5_n13_e26+Pj4k5_n13_e27+Pj4k5_n13_e28+Pj4k5_n13_e29+Pj4k5_n13_e30+Pj4k5_n13_e31+Pj4k5_n13_e32+Pj4k5_n13_e33_38
with open("Pj4k5n13.txt","wb") as f:
    for graph in Pj4k5_n13_final:
      nx.write_graph6(graph,f,header=False) 

#### n=14 case! 

325,895,570 cases to check. Time to plug along.

In [2]:
#Load in n=13
#restarted the kernel to clear cache
Pj4k5_n13=nx.read_graph6("Pj4k5n13.txt")



In [3]:
#Sort Pj4k5_n13 by edge count

#First, we need to identify the possible edge counts for n=13
thirteen_hash=set()
for graph in Pj4k5_n13:
    edge_count=len(graph.edges())
    if edge_count not in thirteen_hash:
        thirteen_hash.add(edge_count)
print(thirteen_hash)
#Next, we sort into a dictionary by edge count
Pj4k5_n13_dict={k:[] for k in thirteen_hash}
for graph in Pj4k5_n13:
    k=len(graph.edges())
    Pj4k5_n13_dict[k].append(graph)

{32, 33, 34, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}


In [4]:
#Now, sort the powerset by how many edges it adds to the G14
n=14
Possible_Edges=[(n-1,k) for k in range(n-1)]
#lower bound is now 3
powerset13=powerset(Possible_Edges,upper_bound=8,lower_bound=3)
#sort into a dictionary by # of edges added
powerset_dict={k:[] for k in range(3,9)} 
for edges in powerset13:
    powerset_dict[len(edges)].append(edges)

#There are 21-34 edged graphs in the G13, and 3-8 being added, so we will be checking graphs with 24 to 42 edges
#for each possible number k for e(G14), we will only check graphs constructed with numbers that sum to k

#Now, for each possible edge count for a G13, we will make a list of tuples 
# that indicated what combos add to that edge count
#so for 24 we have [(21,3)], for 25 we have [(22,3),(21,4)], etc
combos_dict={k:[] for k in range(24,43)}
for G13_size in thirteen_hash:
    for edges_size in range(3,9):
        combos_dict[G13_size+edges_size].append((G13_size,edges_size))

print(combos_dict)

#Counting which cases will be the worst to do
for k in combos_dict:
    count3=0
    for tuple in combos_dict[k]:
        count3+=len(Pj4k5_n13_dict[tuple[0]])*len(powerset_dict[tuple[1]])
    print(k,count3)

{24: [(21, 3)], 25: [(21, 4), (22, 3)], 26: [(21, 5), (22, 4), (23, 3)], 27: [(21, 6), (22, 5), (23, 4), (24, 3)], 28: [(21, 7), (22, 6), (23, 5), (24, 4), (25, 3)], 29: [(21, 8), (22, 7), (23, 6), (24, 5), (25, 4), (26, 3)], 30: [(22, 8), (23, 7), (24, 6), (25, 5), (26, 4), (27, 3)], 31: [(23, 8), (24, 7), (25, 6), (26, 5), (27, 4), (28, 3)], 32: [(24, 8), (25, 7), (26, 6), (27, 5), (28, 4), (29, 3)], 33: [(25, 8), (26, 7), (27, 6), (28, 5), (29, 4), (30, 3)], 34: [(26, 8), (27, 7), (28, 6), (29, 5), (30, 4), (31, 3)], 35: [(32, 3), (27, 8), (28, 7), (29, 6), (30, 5), (31, 4)], 36: [(32, 4), (33, 3), (28, 8), (29, 7), (30, 6), (31, 5)], 37: [(32, 5), (33, 4), (34, 3), (29, 8), (30, 7), (31, 6)], 38: [(32, 6), (33, 5), (34, 4), (30, 8), (31, 7)], 39: [(32, 7), (33, 6), (34, 5), (31, 8)], 40: [(32, 8), (33, 7), (34, 6)], 41: [(33, 8), (34, 7)], 42: [(34, 8)]}
24 286
25 2431
26 15873
27 105248
28 574145
29 2386527
30 7397819
31 17441424
32 32395220
33 48795175
34 60357297
35 60807461
36 

##### e=24-32 case

The only edge cases that I don't want to run with the rest are 33 to 36. So we get the first 9 cases out of the way quickly


In [5]:
#run through 24-32
Pj4k5_n14_e24_32=[]
count=0
start_time=time.time()
for k in range(24,33):
    print(chr(10)+"Now checking graphs with "+str(k)+" edges")
    #grab all pairs i,j that add to k
    combos_to_check=combos_dict[k]
    hash_set_k=set()
    time1=time.time()
    #counts the number of graphs with k vertices checked
    k_count=0
    How_many_to_check=0
    for tuple in combos_to_check:
        How_many_to_check+=len(Pj4k5_n13_dict[tuple[0]])*len(powerset_dict[tuple[1]])
    print("We have",How_many_to_check, "graphs to check")
    for tuple in combos_to_check:
        #all G12s with the desired i value
        print(tuple)
        G13s_to_check=Pj4k5_n13_dict[tuple[0]]
        #all edge sets with the desired j value
        edge_combos=powerset_dict[tuple[1]]
        #iterate through all combos of them
        for edges in edge_combos:
            for G13 in G13s_to_check:
                #create the graph
                G14=G13.copy()
                G14.add_node(n-1)
                G14.add_edges_from(edges)
                #count how many we checked
                count+=1
                k_count+=1
                pn_graph = convertNetworkXGraph(G14)
                cert = pn.certificate(pn_graph)
                if cert not in hash_set_k:
                    #if not already checked, add it and check if it is in J4 K5
                    hash_set_k.add(cert)
                    if not ContainsJ4(G14) and not ContainsAntiK5(G14):
                        #We store if it is in the magical set we are trying to find
                        Pj4k5_n14_e24_32.append(G14)
    elapsedk=time.time()-time1
    elapsedtotal=time.time()-start_time
    print("done with graphs with "+str(k)+" edges")
    print(str(k_count)+" graphs completed in "+str(int(10*elapsedk)/10)+" seconds")
    print(str(len(hash_set_k))+" unique graphs checked")
    print(str(count)+" graphs total completed in "+ str(int(elapsedtotal/6)/10)+" minutes")
    print("Total in P(J4,K5,14,24-32) found:",len(Pj4k5_n14_e24_32))
    #clears the hash set so it frees up space
    del hash_set_k
    gc.collect()
                




Now checking graphs with 24 edges
We have 286 graphs to check
(21, 3)
done with graphs with 24 edges
286 graphs completed in 0.0 seconds
30 unique graphs checked
286 graphs total completed in 0.0 minutes
Total in P(J4,K5,14,24-32) found: 0

Now checking graphs with 25 edges
We have 2431 graphs to check
(21, 4)
(22, 3)
done with graphs with 25 edges
2431 graphs completed in 1.0 seconds
479 unique graphs checked
2717 graphs total completed in 0.0 minutes
Total in P(J4,K5,14,24-32) found: 0

Now checking graphs with 26 edges
We have 15873 graphs to check
(21, 5)
(22, 4)
(23, 3)
done with graphs with 26 edges
15873 graphs completed in 4.9 seconds
6463 unique graphs checked
18590 graphs total completed in 0.1 minutes
Total in P(J4,K5,14,24-32) found: 0

Now checking graphs with 27 edges
We have 105248 graphs to check
(21, 6)
(22, 5)
(23, 4)
(24, 3)
done with graphs with 27 edges
105248 graphs completed in 35.1 seconds
65547 unique graphs checked
123838 graphs total completed in 0.7 minutes

In [6]:
#Save this partial census
with open("Pj4k5n14e24_32.txt","wb") as f:
    for graph in Pj4k5_n14_e24_32:
      nx.write_graph6(graph,f,header=False) 